## Observations

<b>Positive</b>
- Non Null values for all columns except Primary Topic


<b>Negative</b>
- Data types are not correct.

  Session_id should be object

  Visit date should be date_time

- "Traffic Source" has same value but different spellings
- Primary topic has NULL values
- Need to understand what "Page title" and "Actions" are.
- "Primary Topic" has different spellings for same word. There are special characters like (; / 4345 test)
- "Page Title" has data issues. Same word has different spellings.

In [1]:
import pandas as pd

In [2]:
web_path = "../data/Web_Traffic.csv"
web_df = pd.read_csv(web_path)
web_df.head()

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
0,11002649,4,1-Jan-23,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe
1,11002963,5,1-Jan-23,Organic,Srcoll 50%,13,Contact,00v2000000KLfqsAAD,Fund Universe
2,11000521,7,1-Jan-23,Google,Srcoll 50%,3,Home,00v2000000KO68GAAT,Fund Universe
3,11001502,5,1-Jan-23,LinkedIn,Download,14,Contact,00v2000000KrZiaAAF,Fund Universe
4,11001448,6,1-Jan-23,Organic,Login,3,Home,00v2000000ez1ucAAA,Fund Universe


In [3]:
web_df.dtypes

Session ID                  int64
Pageviews                   int64
Visit Date                 object
Traffic Source             object
Actions                    object
Time spent (in minutes)     int64
Page Title                 object
Contact ID                 object
Primary Topic              object
dtype: object

In [4]:
web_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428332 entries, 0 to 428331
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Session ID               428332 non-null  int64 
 1   Pageviews                428332 non-null  int64 
 2   Visit Date               428332 non-null  object
 3   Traffic Source           428174 non-null  object
 4   Actions                  428332 non-null  object
 5   Time spent (in minutes)  428332 non-null  int64 
 6   Page Title               428332 non-null  object
 7   Contact ID               428332 non-null  object
 8   Primary Topic            291885 non-null  object
dtypes: int64(3), object(6)
memory usage: 29.4+ MB


In [5]:
web_df.describe(include = "all")

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
count,4.283320e+05,428332.000000,428332,428174,428332,428332.000000,428332,428332,291885
unique,NaN,NaN,412,6,8,NaN,7,3872,10
top,NaN,NaN,8-Oct-23,LinkedIn,Srcoll 50%,NaN,Home,00v2000000KXVZoAAP,Funds
freq,NaN,NaN,1131,179156,80518,NaN,102632,140,85450
mean,1.100176e+07,4.000712,NaN,NaN,NaN,8.006791,NaN,NaN,NaN
std,9.508609e+02,2.000969,NaN,NaN,NaN,4.320452,NaN,NaN,NaN
min,1.100011e+07,1.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
25%,1.100094e+07,2.000000,NaN,NaN,NaN,4.000000,NaN,NaN,NaN
50%,1.100176e+07,4.000000,NaN,NaN,NaN,8.000000,NaN,NaN,NaN
75%,1.100258e+07,6.000000,NaN,NaN,NaN,12.000000,NaN,NaN,NaN


### Duplicate Analysis

- No duplicates

In [6]:
web_df[web_df.duplicated()]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic


### Column value counts

In [6]:
web_df["Traffic Source"].value_counts()

Traffic Source
LinkedIn    179156
Organic      88277
Youtube      62203
Google       60922
YouTube      30530
organic       7086
Name: count, dtype: int64

In [7]:
web_df["Actions"].value_counts()

Actions
Srcoll 50%     80518
Download       80195
Scroll 25%     53731
Video          53545
Login          53507
CTA            53450
Scroll 100%    26820
Scroll 75%     26566
Name: count, dtype: int64

In [8]:
web_df["Page Title"].value_counts()

Page Title
Home          102632
Funds          85450
Navigation     85356
Strategy       51686
Contact        34595
Invest         34309
funds          34304
Name: count, dtype: int64

In [9]:
web_df[["Page Title", "Actions"]].value_counts().sort_index()

Page Title  Actions    
Contact     CTA             4313
            Download        6505
            Login           4338
            Scroll 100%     2131
            Scroll 25%      4340
            Scroll 75%      2137
            Srcoll 50%      6546
            Video           4285
Funds       CTA            10616
            Download       16149
            Login          10659
            Scroll 100%     5422
            Scroll 25%     10675
            Scroll 75%      5332
            Srcoll 50%     15972
            Video          10625
Home        CTA            12790
            Download       19083
            Login          12804
            Scroll 100%     6388
            Scroll 25%     13012
            Scroll 75%      6364
            Srcoll 50%     19328
            Video          12863
Invest      CTA             4219
            Download        6420
            Login           4312
            Scroll 100%     2173
            Scroll 25%      4303
            Scroll 

In [10]:
web_df["Primary Topic"].value_counts()

Primary Topic
Funds            85450
Navigation       85356
Strategy         51686
Invest           34309
funds            34304
Fund Universe      650
/                   96
;                   32
4345                 1
test                 1
Name: count, dtype: int64

### Primary Topic   / ; 4345 test

- Page Title is mapped to Primary Topic
- All except Home and Contact works fine. They are majorly NULL. Some of them are filled with Fund Universe, /, ; , 4345, test
- test entry can be removed
- Assign "Undefined" topic to Home and Contact Page title

In [11]:
web_df[web_df["Primary Topic"].notnull()][["Page Title", "Primary Topic"]].value_counts()

Page Title  Primary Topic
Funds       Funds            85450
Navigation  Navigation       85356
Strategy    Strategy         51686
Invest      Invest           34309
funds       funds            34304
Home        Fund Universe      490
Contact     Fund Universe      160
Home        /                   74
            ;                   23
Contact     /                   22
            ;                    9
            4345                 1
Home        test                 1
Name: count, dtype: int64

In [12]:
web_df[web_df["Primary Topic"].isnull()]["Page Title"].value_counts()

Page Title
Home       102044
Contact     34403
Name: count, dtype: int64

### Session ID 

In [13]:
web_df.head()

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
0,11002649,4,1-Jan-23,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe
1,11002963,5,1-Jan-23,Organic,Srcoll 50%,13,Contact,00v2000000KLfqsAAD,Fund Universe
2,11000521,7,1-Jan-23,Google,Srcoll 50%,3,Home,00v2000000KO68GAAT,Fund Universe
3,11001502,5,1-Jan-23,LinkedIn,Download,14,Contact,00v2000000KrZiaAAF,Fund Universe
4,11001448,6,1-Jan-23,Organic,Login,3,Home,00v2000000ez1ucAAA,Fund Universe


In [14]:
web_df[web_df["Session ID"] == 11002649]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
0,11002649,4,1-Jan-23,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe
1217,11002649,5,2-Jan-23,LinkedIn,Scroll 100%,7,funds,00v2000000KWVjeAAH,funds
4088,11002649,1,4-Jan-23,Organic,Video,8,funds,00v2000000KLpkHAAT,funds
6631,11002649,4,7-Jan-23,LinkedIn,Scroll 75%,4,Funds,00v2000000KMNBTAA5,Funds
11504,11002649,3,11-Jan-23,LinkedIn,Srcoll 50%,11,funds,00v2000000ZCEVVAA5,funds
...,...,...,...,...,...,...,...,...,...
426263,11002649,4,15-Feb-24,Google,CTA,11,funds,00v2000000KNzJUAA1,funds
426554,11002649,7,15-Feb-24,LinkedIn,CTA,6,Navigation,00v2000000TKvPaAAL,Navigation
427901,11002649,6,16-Feb-24,Organic,Download,1,Funds,00v2000000SJKgSAAX,Funds
427944,11002649,3,16-Feb-24,Organic,CTA,6,Funds,00v2000000OSlV4AAL,Funds


In [19]:
web_df[web_df["Contact ID"] == "00v2000000KM7EdAAL"]

,Session ID,Pageviews,Visit Date,Traffic Source,Actions,Time spent (in minutes),Page Title,Contact ID,Primary Topic
0,11002649,4,1-Jan-23,LinkedIn,Download,9,Contact,00v2000000KM7EdAAL,Fund Universe
6313,11001732,5,6-Jan-23,Google,Scroll 75%,4,Invest,00v2000000KM7EdAAL,Invest
7024,11000241,2,7-Jan-23,Youtube,Download,1,Home,00v2000000KM7EdAAL,NaN
10416,11003116,5,11-Jan-23,LinkedIn,Download,2,Funds,00v2000000KM7EdAAL,Funds
14035,11000165,2,14-Jan-23,LinkedIn,Scroll 75%,9,Home,00v2000000KM7EdAAL,NaN
...,...,...,...,...,...,...,...,...,...
418679,11002626,5,7-Feb-24,YouTube,Download,3,Funds,00v2000000KM7EdAAL,Funds
419187,11000810,6,8-Feb-24,LinkedIn,CTA,8,Navigation,00v2000000KM7EdAAL,Navigation
422591,11001145,4,11-Feb-24,LinkedIn,Video,8,Navigation,00v2000000KM7EdAAL,Navigation
424821,11003049,7,13-Feb-24,Google,Download,13,Home,00v2000000KM7EdAAL,NaN


In [15]:
web_df[web_df["Session ID"] == 11002649]["Contact ID"].value_counts()

Contact ID
00v2000000KZAdeAAH    2
00v2000000KXc97AAD    2
00v2000000gTNqYAAW    2
00v2000000KMF2vAAH    2
00v2000000KrhxSAAR    2
                     ..
00v2000000KXimcAAD    1
00v2000000MibWDAAZ    1
00v2000000M5ajtAAB    1
00v2000000KXvfnAAD    1
00v2000000dyEtdAAE    1
Name: count, Length: 126, dtype: int64

In [16]:
contact_path = "../data/Contact_details.csv"
contact_df = pd.read_csv(contact_path)

In [17]:
unique_contacts = web_df[web_df["Session ID"] == 11002649]["Contact ID"].unique()

contact_df[contact_df["Contact_id"].isin(unique_contacts)]

,Contact Name,Company_name,Contact_id,Local Salutation,Region,Country
18,Abra Boris,Bloomberg,00v2000000Mhvw9AAB,Dear Abra Boris,EMEA,UK
46,Alvarado Bowman,Deloitte & Touche,00v2000000bFleNAAS,Dear Alvarado Bowman,EMEA,United Kingdom
57,Alec Contreras,Diebold Inc.,00v2000000cVOx3AAG,Dear Alec Contreras,EMEA,United Kingdom
112,Amelia Whitley,Kemper Insurance,00v2000000IZ0FkAAL,Dear Amelia Whitley,EMEA,Uk
120,acqueline Barnett,Liberty Mutual Insurance,00v2000000MOJDNAA5,Dear acqueline Barnett,EMEA,Uk
...,...,...,...,...,...,...
3784,Raymond Rollins,HSBC,00v2000000KsAcFAAV,Dear Raymond Rollins,AMER,Canada
3787,Patel Wilder,HSBC,00v2000000KsAjEAAV,Dear Patel Wilder,AMER,Canada
3792,Craig Bates,HSBC,00v2000000KrhxSAAR,Dear Craig Bates,AMER,Canada
3813,Barrera Stokes,HSBC,00v2000000Ka4lrAAB,Dear Barrera Stokes,AMER,Canada
